In [10]:
# Robust .env loader (works from root or /scripts; handles UTF-16 and odd encodings)
import os, json
from pathlib import Path

try:
    from dotenv import load_dotenv, dotenv_values
except Exception:
    load_dotenv = None
    dotenv_values = None

# Resolve repo root and .env path
cwd = Path.cwd().resolve()
repo_root = next((p for p in [cwd, *cwd.parents] if (p / ".git").exists() or p.name == "spending-dashboard"), cwd)
env_path = repo_root / "scripts" / ".env"

print("Looking for .env at:", env_path)
assert env_path.exists(), f".env not found at {env_path}"

def _ensure_utf8(path: Path) -> str:
    raw = path.read_bytes()
    # Detect BOM/UTF-16 and transcode to UTF-8 string
    if raw[:2] in (b"\xff\xfe", b"\xfe\xff"):
        return raw.decode("utf-16")
    try:
        return raw.decode("utf-8")
    except UnicodeDecodeError:
        # Fallback: Latin-1 then re-encode
        return raw.decode("latin-1")

txt = _ensure_utf8(env_path)

# If python-dotenv is available, load via it; else parse manually
loaded = False
if load_dotenv is not None:
    try:
        loaded = load_dotenv(dotenv_path=str(env_path), override=True, encoding="utf-8")
    except TypeError:
        # Older python-dotenv without 'encoding' kw
        loaded = load_dotenv(dotenv_path=str(env_path), override=True)
    if not loaded and dotenv_values is not None:
        vals = dotenv_values(dotenv_path=str(env_path))
        os.environ.update({k:str(v) for k,v in vals.items() if v is not None})
        loaded = True
else:
    # Minimal manual parse as a fallback
    for line in txt.splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if line.lower().startswith("export "):
            line = line[7:].strip()
        if "=" in line:
            k, v = line.split("=", 1)
            os.environ[k.strip()] = v.strip().strip('"').strip("'")
    loaded = True

# Sanity print (masked)
def mask(s): 
    return "<missing>" if not s else (s[:4] + "…" + s[-4:]) if len(s) > 8 else "***"

PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET    = os.getenv("PLAID_SECRET")
PLAID_ENV       = (os.getenv("PLAID_ENV", "production") or "production").strip().lower()
print("Loaded vars →",
      "PLAID_CLIENT_ID:", mask(PLAID_CLIENT_ID),
      "| PLAID_SECRET:", mask(PLAID_SECRET),
      "| PLAID_ENV:", PLAID_ENV)

assert PLAID_CLIENT_ID and PLAID_SECRET, "Still missing PLAID_CLIENT_ID/PLAID_SECRET after load. See checklist below."


Looking for .env at: C:\Users\kosis\Downloads\Automation\spending-dashboard\scripts\.env
Loaded vars → PLAID_CLIENT_ID: 68bb…6689 | PLAID_SECRET: a605…7df5 | PLAID_ENV: production


In [ ]:
# Cell A — start a small Flask app in the background to run Plaid Link in production
import os, json, secrets, threading
from pathlib import Path
from flask import Flask, jsonify, request, send_from_directory
from plaid.api import plaid_api
from plaid.configuration import Configuration, Environment
from plaid.api_client import ApiClient
from plaid.model.link_token_create_request import LinkTokenCreateRequest
from plaid.model.link_token_create_request_user import LinkTokenCreateRequestUser
from plaid.model.products import Products
from plaid.model.country_code import CountryCode
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest

# Resolve repo + token path
cwd = Path.cwd().resolve()
repo_root = next((p for p in [cwd, *cwd.parents] if (p/".git").exists() or p.name=="spending-dashboard"), cwd)
TOKENS_PATH = repo_root / ".state" / "access_tokens.json"
TOKENS_PATH.parent.mkdir(parents=True, exist_ok=True)

# Read env already loaded earlier
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET    = os.getenv("PLAID_SECRET")
assert PLAID_CLIENT_ID and PLAID_SECRET, "Missing PLAID_CLIENT_ID/PLAID_SECRET"

# Plaid client (PRODUCTION)
cfg = Configuration(host=Environment.Production)
cfg.api_key["clientId"] = PLAID_CLIENT_ID
cfg.api_key["secret"]   = PLAID_SECRET
client = plaid_api.PlaidApi(ApiClient(cfg))

app = Flask(__name__, static_folder=str(repo_root / "scripts"))

# Serve the link.html we'll create next
@app.get("/")
def index():
    return send_from_directory(str(repo_root / "scripts"), "link.html")

@app.post("/create_link_token")
def create_link_token():
    user_id = "user-" + secrets.token_hex(8)
    req = LinkTokenCreateRequest(
        user=LinkTokenCreateRequestUser(client_user_id=user_id),
        client_name="Spending Dashboard",
        products=[Products("transactions")],
        country_codes=[CountryCode("US")],
        language="en",
    )
    link_token = client.link_token_create(req).to_dict()["link_token"]
    return jsonify({"link_token": link_token})

@app.post("/exchange_public_token")
def exchange_public_token():
    data = request.get_json(force=True)
    public_token = data["public_token"]
    issuer = (data.get("institution_name") or "Bank").strip() or "Bank"
    access_token = client.item_public_token_exchange(
        ItemPublicTokenExchangeRequest(public_token=public_token)
    ).to_dict()["access_token"]

    # Merge into canonical JSON
    mapping = {}
    if TOKENS_PATH.exists():
        try:
            mapping = json.loads(TOKENS_PATH.read_text(encoding="utf-8")) or {}
            if not isinstance(mapping, dict): mapping = {}
        except Exception:
            mapping = {}
    key = issuer
    i = 2
    while key in mapping and mapping[key] != access_token:
        key = f"{issuer} {i}"; i += 1
    mapping[key] = access_token
    TOKENS_PATH.write_text(json.dumps(mapping, indent=2), encoding="utf-8")
    return jsonify({"ok": True, "issuer": key})

def _run():
    # Don’t use reloader in notebooks
    app.run(host="127.0.0.1", port=5000, debug=False, use_reloader=False)

# Start server in background thread if not already running
if not getattr(app, "_bg_started", False):
    th = threading.Thread(target=_run, daemon=True)
    th.start()
    app._bg_started = True
    print("✅ Flask started at http://127.0.0.1:5000")
else:
    print("Flask already running at http://127.0.0.1:5000")
print("Tokens file:", TOKENS_PATH)


✅ Flask started at http://127.0.0.1:5000
Tokens file: C:\Users\kosis\Downloads\Automation\spending-dashboard\.state\access_tokens.json


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [12]:
# Cell B — create scripts/link.html
from pathlib import Path
html = """<!doctype html>
<html>
  <head><meta charset="utf-8"><title>Plaid Link (Prod)</title></head>
  <body>
    <button id="link-btn">Link a bank</button>
    <pre id="log" style="white-space:pre-wrap"></pre>
    <script src="https://cdn.plaid.com/link/v2/stable/link-initialize.js"></script>
    <script>
      const log = (m) => (document.getElementById('log').textContent += m + "\\n");
      async function openLink() {
        const lt = await fetch("/create_link_token", {method:"POST"}).then(r=>r.json());
        const handler = Plaid.create({
          token: lt.link_token,
          onSuccess: async (public_token, metadata) => {
            log("Exchanging token…");
            const res = await fetch("/exchange_public_token", {
              method: "POST",
              headers: {"Content-Type":"application/json"},
              body: JSON.stringify({
                public_token,
                institution_name: metadata?.institution?.name || "Bank"
              })
            }).then(r=>r.json());
            if (res.ok) log("✅ Saved access_token for: " + res.issuer);
            else log("❌ Exchange failed");
          },
          onExit: (err) => { if (err) log("Exit: " + err.error_code); }
        });
        handler.open();
      }
      document.getElementById('link-btn').onclick = openLink;
    </script>
  </body>
</html>"""
link_path = (Path.cwd() / "link.html")
link_path.write_text(html, encoding="utf-8")
print("✅ Wrote", link_path)
print("Open http://127.0.0.1:5000 in your browser, click 'Link a bank', finish Discover / SSSCU / Petal.")


✅ Wrote c:\Users\kosis\Downloads\Automation\spending-dashboard\scripts\link.html
Open http://127.0.0.1:5000 in your browser, click 'Link a bank', finish Discover / SSSCU / Petal.


127.0.0.1 - - [09/Sep/2025 20:08:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2025 20:08:04] "POST /create_link_token HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2025 20:08:34] "POST /exchange_public_token HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2025 20:08:37] "POST /create_link_token HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2025 20:09:05] "POST /exchange_public_token HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2025 20:09:08] "POST /create_link_token HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2025 20:09:35] "POST /exchange_public_token HTTP/1.1" 200 -


In [13]:
# Verify each access token via /accounts/get
from plaid.model.accounts_get_request import AccountsGetRequest
from plaid.api_client import ApiException
import json

with open(TOKENS_PATH, "r", encoding="utf-8") as f:
    ACCESS_TOKENS = json.load(f)

ok = True
for issuer, tok in ACCESS_TOKENS.items():
    try:
        resp = client.accounts_get(AccountsGetRequest(access_token=tok))
        n = len(resp.to_dict().get("accounts", []))
        print(f"{issuer}: ✅ accounts_get OK ({n} accounts)")
    except ApiException as e:
        ok = False
        print(f"{issuer}: ❌ API {e.status} -> {getattr(e,'body',e)}")
assert ok, "Fix failing issuers before fetching transactions."


Discover: ✅ accounts_get OK (1 accounts)
Petal: ✅ accounts_get OK (1 accounts)
Silver State Schools Credit Union: ✅ accounts_get OK (3 accounts)
